# Bronze Dataset Generation using Wildfire Spatial Data
<font size=3>**Author:** Ashkan Soltanieh<br>
**Date:** Jan. 8, 2022</font>

## Table of Contents

<div class="alert alert-block alert-info mt-20">
    <ul>
        <li><a href="#Overview">Overview</a></li>
        <li><a href="#Approach">Approach</a></li>
        <li><a href="#Metadata">Metadata Review</a></li>
    </ul>
</div>

## Overview
The dataset is subset of *[ABoVE: Wildfire Date of Burning within Fire Scars across Alaska and Canada, 2001-2019](https://daac.ornl.gov/ABOVE/guides/Wildfires_Date_of_Burning.html)* where Alberta and British Colombia related wildfire data has been extracted. All meta data about the tables are all retrieved from the link above.

## Approach
In this notebook, the spatial shape files which are GIS related is converted into pandas dataframe and filtered for Alberta and BC data. There are two wildfire related datasets that we're dealing with here:
1. Fire Area of Burn (AoB) data which are recordered in ABoVE_DoB_2010 to ABoVE_DoB_2019 directories in raw datasets. Geometry data in this table refers to Polygon object which will be used to calculate AoB for silver dataset.
2. Fire characteristics data which are recorded in ABoVE_DoB_MODIS_Active_Fires_2010 to ABoVE_DoB_MODIS_Active_Fires_2019 directories in raw datasets.

Spetial properties of shape files are as follows:<br>
>EPSG: 102001
PROJCS["Canada_Albers_Equal_Area_Conic",
        GEOGCS["GCS_North_American_1983",
        DATUM["D_North_American_1983",
        SPHEROID["GRS_1980",6378137.0,298.257222101]],<br>
PRIMEM["Greenwich",0.0],<br>
UNIT["Degree",0.0174532925199433]],<br>
PROJECTION["Albers"],<br>
PARAMETER["False_Easting",0.0],<br>
PARAMETER["False_Northing",0.0],<br>
PARAMETER["Central_Meridian",-96.0],<br>
PARAMETER["Standard_Parallel_1",50.0],<br>
PARAMETER["Standard_Parallel_2",70.0],<br>
PARAMETER["Latitude_Of_Origin",40.0],<br>
UNIT["Meter",1.0]]<br>

In [4]:
import os
import pandas as pd
import sys
from IPython.display import display

In [5]:
# get data from data/raw folder and clean them
paths_AoB = []
for i in range(0, 10):
    path = os.path.abspath(os.path.join(os.getcwd(), '../data/raw/wildfire/ABoVE_DoB_201' + str(i) + '/ABoVE_DoB_201' + str(i) + '.shp'))
    paths_AoB.append(path)

paths_chracteristics = []
for i in range(0, 10):
    path = os.path.abspath(os.path.join(os.getcwd(), '../data/raw/wildfire/ABoVE_DoB_MODIS_Active_Fires_201' + str(i) + '/ABoVE_DoB_MODIS_Active_Fires_201' + str(i) + '.shp'))
    paths_chracteristics.append(path)

In [6]:
# get the bronze dataframe using custom script
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(),"..","src/data")))
from wildfire import make_bronze_dataframes

gdf_AoB, gdf_characteristics = make_bronze_dataframes(paths_AoB, paths_chracteristics)

In [7]:
# save extracted raw dataframe into bronze dataset
path_AoB = os.path.abspath(
        os.path.join(os.getcwd(), "../data/processed/wildfire/bronze/bronze_AoB.csv"))
path_characteristics = os.path.abspath(
        os.path.join(os.getcwd(), "../data/processed/wildfire/bronze/bronze_chracteristics.csv"))

gdf_AoB.to_csv(path_AoB, index = False)
gdf_characteristics.to_csv(path_characteristics, index = False)

## Metadata

In [8]:
display(gdf_AoB.head())
display(gdf_characteristics.head())

,UID_Fire,FD_Agency,REF_ID,date_src,Year,JD,Map_Date,geometry
43,193,CA,BC-2010-C10060,1AF,2010,152,2010-06-01,"POLYGON Z ((-1688364.429 1766761.124 0.000, -1..."
44,208,CA,BC-2010-C10258,1AF,2010,212,2010-07-31,"POLYGON Z ((-1765477.675 1801382.759 0.000, -1..."
45,215,CA,BC-2010-C10320,1AF,2010,214,2010-08-02,"POLYGON Z ((-1637226.554 1746616.851 0.000, -1..."
46,222,CA,BC-2010-C20018,1AF,2010,187,2010-07-06,"POLYGON Z ((-1764660.362 1676377.401 0.000, -1..."
47,245,CA,BC-2010-C20293,1AF,2010,212,2010-07-31,"POLYGON Z ((-1755143.000 1702585.419 0.000, -1..."


,YYYYMMDD,HHMM,sat,lat,lon,T21,T31,sample,FRP,conf,type,UID_Fire,Status,FD_Agency,REF_ID,geometry,dn
7,20100816,604,T,50.388,-127.290,307.3,293.0,918,9.3,71,0,547,primary,CA,BC-2010-V90985,POINT (-2143720.640 1647561.231),NaN
8,20100721,936,A,49.199,-124.104,317.4,286.1,44,110.9,95,0,541,primary,CA,BC-2010-V70506,POINT (-1992828.781 1429941.640),NaN
9,20100721,1114,A,49.201,-124.111,305.8,285.8,1229,32.3,66,0,541,primary,CA,BC-2010-V70506,POINT (-1993205.618 1430352.170),NaN
10,20100814,615,T,48.492,-124.029,301.9,290.2,548,5.1,45,0,538,primary,CA,BC-2010-V60945,POINT (-2019595.640 1356043.682),NaN
11,20100814,615,T,48.491,-124.043,357.2,294.9,549,84.1,100,0,538,primary,CA,BC-2010-V60945,POINT (-2020590.036 1356363.410),NaN


**Table 1.** Variable name and descriptions for files named ABoVE_DoB_YYYY.zip.
![Raw Area of Burn Table](../references/image/Raw_Area_of_Burn.png)
**Table 2.** Column name and descriptions for files named ABoVE_DoB_MODIS_Active_Fires_YYYY.zip. Columns 1-11 are variables from the original MCD14ML product. Variables 12-16 were added for the Active Fire Product. **Variable 12 is available for only the period 2016–2019.**<br>
![Raw Active Fire Column Description Table](../references/image/Raw_Active_fires.png)